In [1]:
# !git clone --recursive https://github.com/Microsoft/LightGBM
# %cd /content/LightGBM
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package
# !sudo python setup.py install --precompile
# %cd /content

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = "Colab Notebooks/mango_class/"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# this downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/mango_class


In [2]:

# !rm -rf LightGBM
import lightgbm as lgb
# %cd ..
# %cd build
# !cmake -DUSE_GPU=1 ..

In [3]:
import os
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.image as mpimg
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import glob
import numpy as np
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:

import torchvision.models as models
import torch.nn as nn
import time

import copy
from sklearn.model_selection import train_test_split


In [5]:
import torch
USE_GPU = True

# dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)


using device: cuda


In [6]:
X_train = torch.load("dataset/metadata1")
y_train = torch.load("dataset/metalabel1")

X_test = torch.load("dataset/metaX_dev1")
y_test = torch.load("dataset/metay_dev1")

In [7]:
print(X_train.size())
print(y_train.size())

print(X_test.size())
print(y_test.size())

torch.Size([11200, 5376])
torch.Size([11200])
torch.Size([800, 5376])
torch.Size([800])


In [8]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

In [9]:
from sklearn.preprocessing import StandardScaler

X_train = X_train.cpu()
y_train = y_train.cpu()
X_test = X_test.cpu()
y_test = y_test.cpu()

scaler1 = StandardScaler()
scaler1.fit(X_train)
X_train = scaler1.transform(X_train)

scalar2 = StandardScaler()
scalar2.fit(X_test)
X_test = scalar2.transform(X_test)


In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.size())
print(y_test.size())

(11200, 5376)
(800, 5376)
torch.Size([11200])
torch.Size([800])


In [11]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, 
                         C=0.0001 ,
                         multi_class='multinomial',
                         solver='sag',
                        ).fit(X_train, y_train)
# clf.predict(X_test)
y_train_pred = clf.predict(X_train)
acc_test = clf.score(X_test, y_test)
acc_train = clf.score(X_train, y_train)

In [12]:
print(acc_train)
print(acc_test)

0.7419642857142857
0.74125


In [19]:
import joblib
# save model
joblib.dump(clf, 'model_params/stacking2_logistic1.pkl')
# load model
model_load = joblib.load('model_params/stacking2_logistic1.pkl')

In [20]:
acc_test = model_load.score(X_test, y_test)

In [21]:
acc_test

0.74125

In [13]:
metaX_train = clf.predict_proba(X_train)
metay_train = y_train

In [14]:
torch.save(metaX_train, "dataset/metaX_train1_stacking2")
torch.save(metay_train, "dataset/metay_train1_stacking2")

In [15]:
metaX_test = clf.predict_proba(X_test)
metay_test = y_test

In [16]:
torch.save(metaX_test, "dataset/metaX_test1_stacking2")
torch.save(metay_test, "dataset/metay_test1_stacking2")

In [17]:
jiji

NameError: ignored

In [ ]:
# parameters = {
#     'max_depth': [ 6, 7, 9,10],
#     'num_leaves':[ 60, 80, 90, 100],
# }

# gbm = lgb.LGBMClassifier(objective = 'multiclass',
#                           metric = 'multi_logloss',
#                           max_depth = 6,
#                           num_leaves = 60,
#                           learning_rate = 0.1,
#                           feature_fraction = 0.7,
#                           min_child_samples=21,
#                           min_child_weight=0.001,
#                           bagging_fraction = 1,
#                           bagging_freq = 2,
#                           reg_alpha = 0.001,
#                           reg_lambda = 8,
#                           cat_smooth = 0, 
#                           device = 'gpu',
#                           gpu_platform_id = 0, 
#                           gpu_device_id = 0
#                         )
# gsearch = GridSearchCV(gbm, param_grid=parameters, cv=3)
# gsearch.fit(X, y)
# print('参数的最佳取值:{0}'.format(gsearch.best_params_))
# print('最佳模型得分:{0}'.format(gsearch.best_score_))
# print('最佳模型時間:{0}'.format(gsearch.refit_time_))
# print(gsearch.cv_results_['mean_test_score'])
# print(gsearch.cv_results_['params'])

In [ ]:
parameters = {
              # 'device':'gpu',
              'max_depth': [ 3, 5, 7, 9],

              'num_leaves':[31, 63, 127],

              # 'min_data_in_leaf':[20],
              # 'min_sum_hessian_in_leaf':[1e-3],
              
              # 'feature_fraction': [0.6],
              # 'bagging_fraction': [1.0],
              # # # 'bagging_freq': [2, 4, 5, 6, 8],
              # 'lambda_l1': [0.8],
              # 'lambda_l2': [30],

              # 'learning_rate': [0.2],
              # # 'cat_smooth': [10, 15, 20, ]
}

best_lgbm = None
best_acc = 0
f1, f2 = np.meshgrid(parameters['max_depth'],parameters['num_leaves'])
f1 = f1.flat[:]
f2 = f2.flat[:]

for i, j in zip(f1, f2):
  lgbm = lgb.LGBMClassifier(objective = 'muilti_class',
                          metric = 'multi_logloss',
                          max_depth = i,
                          num_leaves = j,
                          learning_rate = 0.1,
                          feature_fraction = 1,
                          min_child_samples=21,
                          min_child_weight=0.001,
                          bagging_fraction = 1,
                          bagging_freq = 2,
                          reg_alpha = 0.001,
                          reg_lambda = 8,
                          cat_smooth = 0, 
                          device = 'gpu',
                          gpu_platform_id = 0, 
                          gpu_device_id = 0
                          )
  lgbm.fit(X_train, y_train)
  
  y_test_pred = torch.tensor(lgbm.predict(X_test))
  y_train_pred = torch.tensor(lgbm.predict(X_train))
  acc = sum(y_test_pred == y_test).item()/y_test.size(0)
  acc_train = sum(y_train_pred == y_train).item()/y_train.size(0)
  print(f'{i}, {j}, acc in dev:{acc}, acc in train:{acc_train}')
  if best_acc <= acc:
    best_lgbm = lgbm
    best_acc = acc
  
print(best_lgbm, acc) 

In [ ]:
kokoko

In [ ]:
type(y)

In [ ]:
print(X_dev.size())
print(y_dev.size())

In [ ]:
import seaborn as sn
cm = np.zeros([3, 3])
for j, i  in zip(y_pred_test,y_test):
    cm[int(i), int(j.item())] += 1
    
df_cm = pd.DataFrame(cm.astype(np.int32),
                     index = [i for i in ["true_A", "true_B", "true_C"]],
                     columns = [i for i in ["pred_A", "pred_B", "pred_C"]])
plt.figure(figsize = (4, 4))
sn.heatmap(df_cm, annot=True, cmap="BuPu")

In [ ]:
cm.numpy().astype(np.int32)

In [ ]:
測試model1的acc

In [ ]:
print(preds1)